In [2]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set_context('notebook')
sns.set_style('darkgrid')
import sklearn as skl
import sklearn.metrics
import sklearn.learning_curve
import sklearn.ensemble
import sklearn.dummy
import sklearn.grid_search
import sklearn.feature_extraction
import sklearn.feature_extraction.text
import numpy as np
import enchant

import sys
sys.path = sys.path + ['..']
import get_airbnb_data
import db
import learning
import imp
imp.reload(learning)
from learning import *

In [3]:
engine = db.create_root_engine()
rawtable = pd.io.sql.read_sql_table('listings', engine, index_col='id')
Xtr, Xte, ytr, yte = get_training_test_set(rawtable, make_features=make_features3, 
                                               categorize_rating=categorize_rating2)

In [4]:
d = enchant.Dict("en_US")
tokenizer = skl.feature_extraction.text.CountVectorizer().build_tokenizer()

In [11]:
from nltk.tag.stanford import NERTagger

In [12]:
st = NERTagger('../stanford-ner-2015-04-20/classifiers/english.all.3class.distsim.crf.ser.gz',
   '../stanford-ner-2015-04-20/stanford-ner.jar')

In [20]:
st.tag('I am going to Mackenzie'.split())

[[('I', 'O'),
  ('am', 'O'),
  ('going', 'O'),
  ('to', 'O'),
  ('Mackenzie', 'PERSON')]]

#TNT Is it better?
Named entity recognition does not seem to be removing a lot of misspellings

In [31]:
{tagged[0]: d.check(tagged[0]) for tagged in st.tag(tokenizer(rawtable.description[2]))[0] 
 if tagged[1] == 'O' and not d.check(tagged[0])}

{'24hr': False,
 'AirBnB': False,
 'BDFM': False,
 'Highline': False,
 'HouseRules': False,
 'Javits': False,
 'NEIGHBOURHOOD': False,
 'NQR': False,
 'Wifi': False,
 'accomodate': False,
 'isn': False,
 'neighbours': False}

In [32]:
{word: d.check(word) for word in tokenizer(rawtable.description[2]) if not d.check(word)}

{'24hr': False,
 'AirBnB': False,
 'BDFM': False,
 'Highline': False,
 'HouseRules': False,
 'Javits': False,
 'Koreatown': False,
 'NEIGHBOURHOOD': False,
 'NQR': False,
 'Wifi': False,
 'accomodate': False,
 'isn': False,
 'neighbours': False}

In [35]:
{tagged[0]: d.check(tagged[0]) for tagged in st.tag(tokenizer(rawtable.description[10]))[0] 
 if tagged[1] == 'O' and not d.check(tagged[0])}

{'Dekalb': False,
 'HouseRules': False,
 'Stuy': False,
 'Wifi': False,
 'aren': False,
 'doesn': False,
 'hasn': False,
 'ing': False,
 'snuck': False}

In [39]:
{word: d.check(word) for word in tokenizer(rawtable.description[10]) if not d.check(word)}

{'Dekalb': False,
 'HouseRules': False,
 'Stuy': False,
 'Wifi': False,
 'aren': False,
 'doesn': False,
 'hasn': False,
 'ing': False,
 'snuck': False}

In [42]:
d.check(tokenizer(rawtable.description[10])[0] )

True

In [67]:
def count_misspellings(text):
    tokenize = skl.feature_extraction.text.CountVectorizer().build_tokenizer()
    return sum(map(lambda word: not d.check(word), tokenize(text)))

In [69]:
count_misspellings(rawtable.description[0])

16

In [61]:
[(word, d.check(word)) for word in tokenizer(rawtable.description[15]) if not d.check(word)]

[('manhattan', False),
 ('cuban', False),
 ('bs', False),
 ('puerto', False),
 ('rican', False),
 ('thai', False),
 ('manhattan', False),
 ('HouseRules', False)]

In [62]:
{word: d.check(word) for word in tokenizer(rawtable.description[15]) if not d.check(word)}

{'HouseRules': False,
 'bs': False,
 'cuban': False,
 'manhattan': False,
 'puerto': False,
 'rican': False,
 'thai': False}

In [56]:
tokenizer(rawtable.description[15])

['Right',
 'in',
 'the',
 'heart',
 'of',
 'manhattan',
 'gastronomic',
 'and',
 'cultural',
 'center',
 'of',
 'the',
 'east',
 'village',
 'right',
 'in',
 'the',
 'middle',
 'of',
 'all',
 'the',
 'action',
 'Finally',
 'room',
 'not',
 'the',
 'size',
 'of',
 'closet',
 'in',
 'Manhattan',
 'but',
 'very',
 'spacious',
 'area',
 'with',
 'high',
 'ceilings',
 'The',
 'apartment',
 'is',
 'located',
 'in',
 'the',
 'middle',
 'of',
 'the',
 'historic',
 'neighborhood',
 'known',
 'as',
 'the',
 'east',
 'village',
 'you',
 'will',
 'be',
 'surrounded',
 'by',
 'restaurants',
 'lounges',
 'and',
 'bars',
 'with',
 'an',
 'incredible',
 'diversity',
 'of',
 'cuisines',
 'from',
 'all',
 'over',
 'the',
 'world',
 'going',
 'from',
 'home',
 'style',
 'affordable',
 'to',
 'outright',
 'high',
 'end',
 'fine',
 'dining',
 'Bars',
 'that',
 'range',
 'from',
 '1940',
 'cuban',
 'themed',
 'exclusive',
 'speakeasy',
 'to',
 'no',
 'bs',
 'dive',
 'bars',
 'This',
 'is',
 'the',
 'apartme

##SKlearn's tokenizer seems to be better than enchant's
e.g. "181th"

In [79]:
from enchant.checker import SpellChecker
chkr = SpellChecker("en_US")
chkr.set_text(rawtable.description[0])
for word in chkr: print(word.word)

WiFi
Tryon
Tryon
Jumel
Inwood
WiFi
Tryon
Tryon
Jumel
HouseRules
airbnb
airbnb


In [77]:
[(word, d.check(word)) for word in tokenizer(rawtable.description[0]) if not d.check(word)]

[('WiFi', False),
 ('Tryon', False),
 ('181th', False),
 ('42th', False),
 ('Tryon', False),
 ('Jumel', False),
 ('Inwood', False),
 ('WiFi', False),
 ('Tryon', False),
 ('181th', False),
 ('42th', False),
 ('Tryon', False),
 ('Jumel', False),
 ('HouseRules', False),
 ('airbnb', False),
 ('airbnb', False)]